<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [6]</a>'.</span>

<< [Search-11-LinearProgramming](Search-11-LinearProgramming.ipynb) | [Index](../README.md) | [App-1-NQueens](../Applications/App-1-NQueens.ipynb) >>

# Search-12 : Automates Symboliques avec Z3

**Navigation** : [Index](../README.md) | [Suivant >>](../Applications/App-1-NQueens.ipynb)

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. **Comprendre** - Comprendre la difference entre automates finis et symboliques
2. **Appliquer** - Appliquer automata-lib pour les automates finis
3. **Implementer** - Implementer des automates symboliques avec Z3
4. **Resoudre** - Resoudre des problemes de verification avec automates symboliques

### Prerequis
- [Search-1-StateSpace](Search-1-StateSpace.ipynb) - Notion d'espace d'etats
- [SymbolicAI/Sudoku-4-Z3](../../SymbolicAI/SymbolicAI/Sudoku-4-Z3.ipynb) - Bases de Z3

### Duree estimee : 2 heures

## 1. Introduction aux Automates

### 1.1 Qu'est-ce qu'un automate ?

Un **automate** est un modele mathematique de calcul qui consiste en :

| Composant | Description | Notation |
|-----------|-------------|----------|
| **Etats** | Configurations possibles | $Q = \{q_0, q_1, ...\}$ |
| **Alphabet** | Symboles d'entree | $\Sigma = \{a, b, ...\}$ |
| **Transitions** | Regles de passage entre etats | $\delta : Q \times \Sigma \to Q$ |
| **Etat initial** | Point de depart | $q_0 \in Q$ |
| **Etats finaux** | Etats d'acceptation | $F \subseteq Q$ |

Un automate **reconnait** un mot si, en partant de l'etat initial et en suivant les transitions, on atteint un etat final.

### 1.2 Types d'automates finis

**DFA** (Deterministic Finite Automaton) :
- Pour chaque etat et symbole, **exactement une** transition
- Deterministe : pas d'ambiguite

**NFA** (Non-deterministic Finite Automaton) :
- Pour chaque etat et symbole, **zero, une ou plusieurs** transitions
- Peut avoir des transitions epsilon ($\varepsilon$) sans consommation de symbole

> **Theoreme de Kleene** : Tout langage reconnu par un automate fini est regulier et reciproquement.

> **Theoreme de Rabin-Scott** : NFA et DFA reconnaissent les memes langages (NFA peut etre converti en DFA).

### 1.3 Exemple introductif - Reconnaissance de "ab"

Soit l'automate qui reconnait les mots contenant exactement la sequence "ab" :

```
      a         b
q0 -----> q1 -----> q2 (final)
^        |         |
|        | a       | a,b
+--------+---------+
```

- **q0** : etat initial, n'a pas vu "a"
- **q1** : a vu "a", attend "b"
- **q2** : a vu "ab", etat final

Mots acceptes : "ab", "aab", "abab", "cab", ...

Mots rejetes : "", "a", "b", "ba", "aa", ...

### 1.4 Limites des automates finis classiques

Les automates finis classiques souffrent d'une limitation majeure : **l'explosion d'etats**.

**Exemple** : Automate pour entiers 32-bit

- Alphabet : $\{0, 1\}$ (bits)
- Mot : 32 bits representant un entier
- Probleme : Reconnaître les entiers entre 1000 et 2000

**Approche naive** :
- Il faut $2^{32} \approx 4$ milliards d'etats pour representer tous les entiers possibles
- L'automate devient impossible a manipuler

**Solution** : Utiliser des **automates symboliques** avec des predicats logiques au lieu de transitions explicites.

## 2. Automates Finis avec automata-lib

### 2.1 Installation et importation

La librairie `automata-lib` permet de manipuler facilement des automates finis en Python.

In [1]:
# Installation de automata-lib
!pip install -q automata-lib>=9.2.0


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import sys
sys.path.insert(0, '..')

from automata.fa.nfa import NFA
from automata.fa.dfa import DFA
from typing import Set, Dict, List, Tuple

print("Bibliotheques importees :")
print(f"  automata-lib : NFA, DFA")
print("Environnement pret.")

Bibliotheques importees :
  automata-lib : NFA, DFA
Environnement pret.


### 2.2 Creation d'un NFA - Reconnaissance de "ab"

Creons un NFA qui reconnait les mots contenant la sequence "ab".

In [3]:
# NFA pour reconnaissance de "ab"
nfa_ab = NFA(
    states={'q0', 'q1', 'q2'},
    input_symbols={'a', 'b'},
    transitions={
        'q0': {'a': {'q0', 'q1'}},  # Reste en q0 ou va en q1
        'q1': {'b': {'q2'}},        # Si on voit b apres a, va en q2
        'q2': {'a': {'q2'}, 'b': {'q2'}}  # Reste en q2 (final)
    },
    initial_state='q0',
    final_states={'q2'}
)

# Test de quelques mots
test_words = ['ab', 'aab', 'abab', 'a', 'b', 'ba', 'aa']

print("NFA pour reconnaissance de 'ab'\n")
print(f"Etats : {nfa_ab.states}")
print(f"Alphabet : {nfa_ab.input_symbols}")
print(f"Etat initial : {nfa_ab.initial_state}")
print(f"Etats finaux : {nfa_ab.final_states}")
print()
print("Tests d'acceptation :")
print("-" * 40)
for word in test_words:
    try:
        accepted = nfa_ab.accepts_input(word)
        status = "✓ Accepte" if accepted else "✗ Rejete"
        print(f"  '{word}': {status}")
    except Exception as e:
        print(f"  '{word}': Erreur - {e}")

NFA pour reconnaissance de 'ab'

Etats : frozenset({'q0', 'q2', 'q1'})
Alphabet : frozenset({'b', 'a'})
Etat initial : q0
Etats finaux : frozenset({'q2'})

Tests d'acceptation :
----------------------------------------
  'ab': ✓ Accepte
  'aab': ✓ Accepte
  'abab': ✓ Accepte
  'a': ✗ Rejete
  'b': ✗ Rejete
  'ba': ✗ Rejete
  'aa': ✗ Rejete


### Interpretation : NFA pour reconnaissance de "ab"

**Sortie obtenue** : Le NFA accepte les mots contenant la sequence "ab".

| Mot | Accepte | Explication |
|-----|---------|-------------|
| 'ab' | Oui | Contient "ab" |
| 'aab' | Oui | Contient "ab" (positions 2-3) |
| 'abab' | Oui | Contient "ab" (plusieurs fois) |
| 'a' | Non | Ne contient pas "ab" |
| 'b' | Non | Ne contient pas "ab" |
| 'ba' | Non | Contient "ba", pas "ab" |
| 'aa' | Non | Ne contient pas "ab" |

**Points cles** :
1. Le NFD utilise le **non-determinisme** : depuis q0 avec 'a', on peut rester en q0 OU aller en q1
2. Une fois en q2 (etat final), on y reste quel que soit le symbole
3. Le mot est accepte si **au moins un** chemin mene a un etat final

### 2.3 Creation d'un DFA - Mots avec nombre pair de 'a'

Creons un DFA qui reconnait les mots contenant un nombre pair de 'a'.

In [4]:
# DFA pour nombre pair de 'a'
dfa_even_a = DFA(
    states={'q_even', 'q_odd'},
    input_symbols={'a', 'b'},
    transitions={
        'q_even': {'a': 'q_odd', 'b': 'q_even'},  # a change la parite, b non
        'q_odd': {'a': 'q_even', 'b': 'q_odd'}    # a change la parite, b non
    },
    initial_state='q_even',
    final_states={'q_even'}
)

# Tests
test_words = ['', 'a', 'aa', 'aaa', 'b', 'ab', 'aba', 'bab']

print("DFA pour nombre pair de 'a'\n")
print("Schema des transitions :")
print("  q_even --a--> q_odd")
print("  q_even --b--> q_even (final)")
print("  q_odd  --a--> q_even (final)")
print("  q_odd  --b--> q_odd")
print()
print("Tests d'acceptation :")
print("-" * 50)
for word in test_words:
    count_a = word.count('a')
    accepted = dfa_even_a.accepts_input(word)
    status = "✓ Pair" if accepted else "✗ Impair"
    print(f"  '{word}': {status} ({count_a} 'a')")

DFA pour nombre pair de 'a'

Schema des transitions :
  q_even --a--> q_odd
  q_even --b--> q_even (final)
  q_odd  --a--> q_even (final)
  q_odd  --b--> q_odd

Tests d'acceptation :
--------------------------------------------------
  '': ✓ Pair (0 'a')
  'a': ✗ Impair (1 'a')
  'aa': ✓ Pair (2 'a')
  'aaa': ✗ Impair (3 'a')
  'b': ✓ Pair (0 'a')
  'ab': ✗ Impair (1 'a')
  'aba': ✓ Pair (2 'a')
  'bab': ✗ Impair (1 'a')


### Interpretation : DFA pour nombre pair de 'a'

**Sortie obtenue** : Le DFA accepte uniquement les mots avec un nombre pair de 'a'.

| Mot | Nombre de 'a' | Accepte | Etat final |
|-----|--------------|---------|-----------|
| '' | 0 | Oui | q_even |
| 'a' | 1 | Non | q_odd |
| 'aa' | 2 | Oui | q_even |
| 'aaa' | 3 | Non | q_odd |
| 'b' | 0 | Oui | q_even |
| 'ab' | 1 | Non | q_odd |

**Points cles** :
1. Le DFA est **deterministe** : depuis chaque etat avec chaque symbole, il y a exactement une transition
2. L'etat 'q_even' memorise si le nombre de 'a' vu est pair
3. Le symbole 'b' ne change pas l'etat (n'affecte pas la parite de 'a')

> **Remarque** : Le mot vide (epsilon) est accepte car 0 est un nombre pair.

### 2.4 Operations sur les automates

Les automates (et les langages reguliers) supportent plusieurs operations classiques.

In [5]:
print("Operations sur les automates\n")
print("1. UNION : L1 ∪ L2")
print("   - Reconnaît les mots acceptes par L1 OU L2")
print()
print("2. INTERSECTION : L1 ∩ L2")
print("   - Reconnaît les mots acceptes par L1 ET L2")
print()
print("3. COMPLEMENT : L^c = Σ* \ L")
print("   - Reconnaît les mots NON acceptes par L")
print()
print("4. PRODUIT (Concatenation) : L1 · L2")
print("   - Reconnaît les mots w = w1·w2 ou w1∈L1 et w2∈L2")
print()
print("5. ETOILE (Kleene) : L*")
print("   - Reconnaît les repetitions (y compris mot vide)")

# Exemple avec automata-lib
print("\nExemple : Union avec automata-lib")
print("Note : automata-lib ne fournit pas d'operation d'union directe,")
print("      mais on peut construire manuellement l'automate resultant.")

Operations sur les automates

1. UNION : L1 ∪ L2
   - Reconnaît les mots acceptes par L1 OU L2

2. INTERSECTION : L1 ∩ L2
   - Reconnaît les mots acceptes par L1 ET L2

3. COMPLEMENT : L^c = Σ* \ L
   - Reconnaît les mots NON acceptes par L

4. PRODUIT (Concatenation) : L1 · L2
   - Reconnaît les mots w = w1·w2 ou w1∈L1 et w2∈L2

5. ETOILE (Kleene) : L*
   - Reconnaît les repetitions (y compris mot vide)

Exemple : Union avec automata-lib
Note : automata-lib ne fournit pas d'operation d'union directe,
      mais on peut construire manuellement l'automate resultant.


### 2.5 Limitation d'automata-lib

**Alphabet fini**

La limitation principale d'`automata-lib` (et des automates finis en general) est que l'alphabet doit etre **fini et explicite**.

**Exemple** : Si on veut travailler avec des entiers 32-bit
- Il faudrait un alphabet de taille $2^{32}$ (impossible)
- L'automate aurait des milliards d'etats

**Solution** : Les **automates symboliques** utilisent des predicats logiques pour representer des ensembles infinis de symboles.

### 2.6 Visualisation d'automates

Visualisons nos automates avec graphviz.

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [6]:
try:
    import graphviz
    HAS_GRAPHVIZ = True
except ImportError:
    HAS_GRAPHVIZ = False
    print("graphviz non disponible. Installation : pip install graphviz")

def visualize_dfa(dfa: DFA, name: str = "DFA"):
    """Visualise un DFA avec graphviz."""
    if not HAS_GRAPHVIZ:
        print(f"Visualisation non disponible (graphviz manquant)")
        return
    
    try:
        dot = graphviz.Digraph(comment=name)
        dot.attr(rankdir='LR')
        
        # Etat initial (fleche entrante)
        dot.node('invisible', shape='point', width='0')
        dot.edge('invisible', dfa.initial_state)
        
        # Etats finaux (double cercle)
        for state in dfa.states:
            if state in dfa.final_states:
                dot.node(state, shape='doublecircle', peripheries='2')
            else:
                dot.node(state, shape='circle')
        
        # Transitions
        for state in dfa.states:
            for symbol in dfa.input_symbols:
                next_state = dfa.transitions[state][symbol]
                dot.edge(state, next_state, label=symbol)
        
        return dot
    except Exception as e:
        print(f"Erreur lors de la visualisation: {e}")
        return None

# Visualisation du DFA "nombre pair de 'a'"
if HAS_GRAPHVIZ:
    try:
        dot = visualize_dfa(dfa_even_a, "DFA: Nombre pair de 'a'")
        # Try display if in Jupyter, otherwise just render
        try:
            from IPython.display import display
            display(dot)
        except ImportError:
            print("Graph viz cree mais display non disponible (pas dans Jupyter)")
            print(dot.source if hasattr(dot, 'source') else 'Dot object created')
    except Exception as e:
        print(f"Erreur: {e}")
else:
    print("\nRepresentation textuelle du DFA 'nombre pair de a' :")
    print("       a         ")
    print("  q_even <--> q_odd")
    print("       | ^       ")
    print("       b |       ")
    print("       v |       ")
    print("     q_even (final)")

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

## 3. Introduction aux Automates Symboliques

### 3.1 Definition

Un **automate symbolique** generalise les automates finis en remplaçant les transitions sur des symboles par des transitions sur des **predicats**.

**Automate fini classique** :
$$\delta : Q \times \Sigma \to Q$$

**Automate symbolique** :
$$\delta : Q \times \Phi \to Q$$

Ou $\Phi$ est un ensemble de **predicats logiques** sur l'alphabet.

**Predicat** : Une formule logique qui est vraie pour certaines valeurs de l'alphabet.

**Exemples de predicats** :
- $x > 0$ : "x est strictement positif"
- $10 \leq x \leq 100$ : "x est dans l'intervalle [10, 100]"
- $x \mod 2 = 0$ : "x est pair"
- $x = y$ : "x est egal a y"

### 3.2 Alphabet infini ou tres grand

Les automates symboliques sont particulirement utiles lorsque :

| Situation | Exemple | Pourquoi symbolique ? |
|-----------|---------|-----------------------|
| Alphabet infini | Entiers, rationnels | Impossible d'enumerer tous les symboles |
| Alphabet tres grand | Entiers 32-bit | $2^{32}$ symboles = explosion d'etats |
| Structure de donnees | Tableaux, arbres | Predicats sur la structure |
| Types de donnees | Entiers, strings | Predicats selon le type |

### 3.3 Predicats comme formules Z3

Nous utiliserons **Z3** pour representer et evaluer les predicats logiques.

**Theorie des predicats** avec Z3 :
- **Arithmetique entiere** : Int, operations +, -, *, /, modulo
- **Bit-vectors** : BitVec, operations bit a bit
- **Logique** : And, Or, Not, Implies
- **Quantificateurs** : ForAll, Exists

### 3.4 Comparaison Automate Fini vs Symbolique

**Exemple** : Reconnaître les entiers pairs entre 10 et 100

**Automate fini (impraticable)** :
- 91 etats (un pour chaque valeur 10, 12, 14, ..., 100)
- 91 transitions (une par valeur paire)

**Automate symbolique** :
- 1 etat (ou 2 etats si on veut separer accept/reject)
- 1 transition avec predicat : $x \geq 10 \land x \leq 100 \land x \mod 2 = 0$

| Aspect | Automate Fini | Automate Symbolique |
|--------|---------------|---------------------|
| Alphabet | Fini, explicite | Infini ou implicite |
| Transitions | $\delta(q, a)$ | $\delta(q, \phi(x))$ |
| Complexite | Explosion d'etats | Taille raisonnable |
| Decision | SAT en temps lineaire | SAT via SMT solver |
| Expressivite | Langages reguliers | Langages avec predicats |

## 4. Automates Symboliques avec Z3

### 4.1 Installation de Z3

In [7]:
# Z3 est deja installe (voir requirements.txt)
# Installation si necessaire :
# !pip install -q z3-solver>=4.13

In [8]:
from z3 import *

print("Z3 importe avec succes.")
print(f"Version Z3 : {get_version()}")
print()
print("Types de variables disponibles :")
print("  - Int    : Entiers mathematiques (infinis)")
print("  - BitVec : Vecteurs de bits (taille fixe)")
print("  - Bool   : Booleens")
print("  - Real   : Nombres reels")

Z3 importe avec succes.
Version Z3 : (4, 15, 4, 0)

Types de variables disponibles :
  - Int    : Entiers mathematiques (infinis)
  - BitVec : Vecteurs de bits (taille fixe)
  - Bool   : Booleens
  - Real   : Nombres reels


### 4.2 Predicats Symboliques avec Z3

Commencons par explorer les predicats de base avec Z3.

In [9]:
# Exemple de predicats Z3
print("Predicats symboliques avec Z3\n")
print("1. Variable entiere :")
x = Int('x')
print(f"   x = {x}")
print(f"   Type : {x.sort()}")

print("\n2. Predicats simples :")
predicates = [
    (x > 0, "x > 0"),
    (x < 100, "x < 100"),
    (x >= 10, "x >= 10"),
    (x <= 50, "x <= 50"),
    (x % 2 == 0, "x est pair"),
]

for pred, desc in predicates:
    print(f"   {desc:15s} -> {pred}")

print("\n3. Predicats composes :")
pred_and = And(x >= 10, x <= 100)
print(f"   10 <= x <= 100  : {pred_and}")

pred_or = Or(x < 0, x > 100)
print(f"   x < 0 ou x > 100 : {pred_or}")

pred_even = x % 2 == 0
print(f"   x est pair       : {pred_even}")

print("\n4. Evaluation de predicats :")
s = Solver()

# Test : est-ce que x=50 satisfait "x >= 10 et x <= 100" ?
s.add(pred_and)
s.add(x == 50)
print(f"   x=50 satisfait '10 <= x <= 100' ? {s.check() == sat}")

Predicats symboliques avec Z3

1. Variable entiere :
   x = x
   Type : Int

2. Predicats simples :
   x > 0           -> x > 0
   x < 100         -> x < 100
   x >= 10         -> x >= 10
   x <= 50         -> x <= 50
   x est pair      -> x%2 == 0

3. Predicats composes :
   10 <= x <= 100  : And(x >= 10, x <= 100)
   x < 0 ou x > 100 : Or(x < 0, x > 100)
   x est pair       : x%2 == 0

4. Evaluation de predicats :
   x=50 satisfait '10 <= x <= 100' ? True


### Interpretation : Predicats Z3

**Sortie obtenue** : Z3 permet de creer des predicats logiques sur des variables entieres.

**Composants d'un predicat** :
- **Variable** : `Int('x')` cree une variable entiere symbolique
- **Operateurs de comparaison** : `>`, `<`, `>=`, `<=`, `==`
- **Operateurs logiques** : `And`, `Or`, `Not`
- **Operateurs arithmetiques** : `+`, `-`, `*`, `/`, `%`

**Predicat compose** : `And(x >= 10, x <= 100)` represente la conjonction de deux contraintes.

**Evaluation** : Z3 peut determiner si un predicat est satisfiable (SAT) ou non (UNSAT).

### 4.3 Classe SymbolicAutomaton

Implementons maintenant une classe pour les automates symboliques.

In [10]:
class SymbolicAutomaton:
    """
    Automate symbolique avec predicats Z3.
    
    Chaque transition est etiquetee par un predicat logique
    plutot que par un symbole explicite.
    """
    
    def __init__(self, name: str = "SymbolicAutomaton"):
        self.name = name
        self.states = set()           # Ensemble des etats
        self.transitions = []         # Liste de (from_state, to_state, predicate)
        self.initial_state = None     # Etat initial
        self.final_states = set()     # Etats finaux
        self.context = None           # Contexte Z3
    
    def add_state(self, state: str, is_initial: bool = False, is_final: bool = False):
        """Ajoute un etat a l'automate."""
        self.states.add(state)
        if is_initial:
            if self.initial_state is not None:
                raise ValueError(f"Etat initial deja defini : {self.initial_state}")
            self.initial_state = state
        if is_final:
            self.final_states.add(state)
        return self
    
    def add_transition(self, from_state: str, to_state: str, predicate):
        """Ajoute une transition etiquetee par un predicat Z3."""
        if from_state not in self.states:
            raise ValueError(f"Etat source inconnu : {from_state}")
        if to_state not in self.states:
            raise ValueError(f"Etat destination inconnu : {to_state}")
        self.transitions.append((from_state, to_state, predicate))
        return self
    
    def accepts(self, input_value: int, variable_name: str = 'x') -> bool:
        """
        Verifie si l'automate accepte une valeur d'entree.
        
        Args:
            input_value: La valeur a tester
            variable_name: Nom de la variable dans les predicats (defaut: 'x')
        
        Returns:
            True si la valeur est acceptee, False sinon
        """
        if self.initial_state is None:
            raise ValueError("Pas d'etat initial defini")
        
        # Creer un solver Z3
        s = Solver()
        
        # Variable pour l'entree
        x = Int(variable_name)
        
        # Etat courant
        current = self.initial_state
        
        # Trouver une transition dont le predicat est satisfait
        for from_state, to_state, predicate in self.transitions:
            if from_state == current:
                # Ajouter le predicat et la valeur d'entree
                s.add(predicate)
                s.add(x == input_value)
                
                # Verifier la satisfiabilite
                if s.check() == sat:
                    current = to_state
                    if current in self.final_states:
                        return True
                    
                    # Continuer depuis le nouvel etat
                    s = Solver()
                    break
        
        return current in self.final_states
    
    def find_accepting_values(self, variable_name: str = 'x', 
                               min_val: int = -100, max_val: int = 100) -> List[int]:
        """
        Trouve toutes les valeurs acceptees dans une plage donnee.
        
        Args:
            variable_name: Nom de la variable dans les predicats
            min_val: Borne inferieure de la recherche
            max_val: Borne superieure de la recherche
        
        Returns:
            Liste des valeurs acceptees
        """
        accepting = []
        for val in range(min_val, max_val + 1):
            if self.accepts(val, variable_name):
                accepting.append(val)
        return accepting
    
    def __repr__(self):
        return (f"{self.name}(states={len(self.states)}, "
                f"transitions={len(self.transitions)}, "
                f"initial={self.initial_state}, "
                f"final={len(self.final_states)})")

print("Classe SymbolicAutomaton definie.")

Classe SymbolicAutomaton definie.


### 4.4 Exemple 1 : Automate de Plage [10, 100]

Creons un automate symbolique qui reconnait les entiers entre 10 et 100.

In [11]:
# Automate pour l'intervalle [10, 100]
automaton_range = SymbolicAutomaton("RangeAutomaton")

# Definir les etats
automaton_range.add_state('q0', is_initial=True)   # Etat initial
automaton_range.add_state('q1', is_final=True)    # Etat final (accepte)

# Predicat pour l'intervalle [10, 100]
x = Int('x')
predicate_in_range = And(x >= 10, x <= 100)

# Transition : si x est dans [10, 100], aller a l'etat final
automaton_range.add_transition('q0', 'q1', predicate_in_range)

# Affichage
print("Automate symbolique pour l'intervalle [10, 100]\n")
print(automaton_range)
print(f"\nTransitions :")
for src, dst, pred in automaton_range.transitions:
    print(f"  {src} --[{pred}]--> {dst}")

# Tests
test_values = [0, 5, 10, 50, 100, 101, 150]
print("\nTests d'acceptation :")
print("-" * 50)
for val in test_values:
    accepted = automaton_range.accepts(val)
    status = "✓ Accepte" if accepted else "✗ Rejete"
    print(f"  {val:4d} : {status}")

Automate symbolique pour l'intervalle [10, 100]

RangeAutomaton(states=2, transitions=1, initial=q0, final=1)

Transitions :
  q0 --[And(x >= 10, x <= 100)]--> q1

Tests d'acceptation :
--------------------------------------------------
     0 : ✗ Rejete
     5 : ✗ Rejete
    10 : ✓ Accepte
    50 : ✓ Accepte
   100 : ✓ Accepte
   101 : ✗ Rejete
   150 : ✗ Rejete


### Interpretation : Automate de Plage

**Sortie obtenue** : L'automate accepte uniquement les valeurs dans [10, 100].

| Valeur | Accepte | Explication |
|--------|---------|-------------|
| 0 | Non | < 10 |
| 5 | Non | < 10 |
| 10 | Oui | = 10 (borne incluse) |
| 50 | Oui | Dans l'intervalle |
| 100 | Oui | = 100 (borne incluse) |
| 101 | Non | > 100 |
| 150 | Non | > 100 |

**Points cles** :
1. L'automate n'a que **2 etats** (contre 91 pour un automate fini classique)
2. La transition utilise un **predicat compose** : $10 \leq x \leq 100$
3. Le predicat est evaluable pour **n'importe quel entier** (alphabet infini)

### 4.5 Exemple 2 : Automate pour Nombres Pairs

Creons un automate qui reconnait les nombres pairs.

In [12]:
# Automate pour nombres pairs
automaton_even = SymbolicAutomaton("EvenAutomaton")

# Definir les etats
automaton_even.add_state('q0', is_initial=True)   # Etat initial
automaton_even.add_state('q1', is_final=True)    # Etat final (accepte)

# Predicat : x est pair (x % 2 == 0)
x = Int('x')
predicate_even = x % 2 == 0

# Transition
automaton_even.add_transition('q0', 'q1', predicate_even)

# Tests
test_values = list(range(-5, 11))

print("Automate symbolique pour nombres pairs\n")
print("Predicat : x % 2 == 0")
print()
print("Tests d'acceptation :")
print("-" * 40)
for val in test_values:
    accepted = automaton_even.accepts(val)
    status = "Pair" if accepted else "Impair"
    print(f"  {val:3d} : {status}")

Automate symbolique pour nombres pairs

Predicat : x % 2 == 0

Tests d'acceptation :
----------------------------------------
   -5 : Impair
   -4 : Pair
   -3 : Impair
   -2 : Pair
   -1 : Impair
    0 : Pair
    1 : Impair
    2 : Pair
    3 : Impair
    4 : Pair
    5 : Impair
    6 : Pair
    7 : Impair
    8 : Pair
    9 : Impair
   10 : Pair


### Interpretation : Automate pour Nombres Pairs

**Sortie obtenue** : L'automate accepte uniquement les nombres pairs (positifs et negatifs).

**Predicat** : `x % 2 == 0` utilise l'operateur modulo pour tester la parite.

| Valeur | Pair/Impair | Accepte |
|--------|-------------|---------|
| -5 | Impair | Non |
| -4 | Pair | Oui |
| -2 | Pair | Oui |
| 0 | Pair | Oui |
| 1 | Impair | Non |
| 2 | Pair | Oui |

**Note** : Le predicat fonctionne pour les entiers negatifs aussi, car Z3 utilise l'arithmetique entiere mathematique.

### 4.6 Exemple 3 : Automate pour Nombres Positifs Multiples de 5

Combinons plusieurs contraintes : positifs ET multiples de 5.

In [13]:
# Automate pour nombres positifs multiples de 5
automaton_pos_mult5 = SymbolicAutomaton("PositiveMultipleOf5")

# Definir les etats
automaton_pos_mult5.add_state('q0', is_initial=True)
automaton_pos_mult5.add_state('q1', is_final=True)

# Predicat compose : x > 0 ET x % 5 == 0
x = Int('x')
predicate = And(x > 0, x % 5 == 0)

automaton_pos_mult5.add_transition('q0', 'q1', predicate)

# Tests
test_values = list(range(-10, 26))

print("Automate symbolique pour nombres positifs multiples de 5\n")
print("Predicat : x > 0 AND x % 5 == 0")
print()
print("Tests d'acceptation :")
print("-" * 45)
for val in test_values:
    accepted = automaton_pos_mult5.accepts(val)
    if accepted:
        print(f"  {val:3d} : ✓ Accepte (positif et multiple de 5)")

# Afficher toutes les valeurs acceptees dans une plage
accepting_vals = automaton_pos_mult5.find_accepting_values(min_val=-50, max_val=50)
print(f"\nValeurs acceptees dans [-50, 50] : {accepting_vals}")

Automate symbolique pour nombres positifs multiples de 5

Predicat : x > 0 AND x % 5 == 0

Tests d'acceptation :
---------------------------------------------
    5 : ✓ Accepte (positif et multiple de 5)
   10 : ✓ Accepte (positif et multiple de 5)
   15 : ✓ Accepte (positif et multiple de 5)
   20 : ✓ Accepte (positif et multiple de 5)
   25 : ✓ Accepte (positif et multiple de 5)



Valeurs acceptees dans [-50, 50] : [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]


### Interpretation : Automate Multiples de 5

**Sortie obtenue** : L'automate accepte les nombres strictement positifs divisibles par 5.

**Predicat compose** : `And(x > 0, x % 5 == 0)` combine deux contraintes.

**Valeurs acceptees** : 5, 10, 15, 20, 25, ...

**Valeurs rejetees** :
- Nombres negatifs (-5, -10, ...) : echec sur `x > 0`
- Zero : echec sur `x > 0`
- Nombres non divisibles par 5 : echec sur `x % 5 == 0`

**Point cle** : La combinaison de predicats permet d'exprimer des conditions complexes de maniere concise.

### 4.7 Operations sur Automates Symboliques

Implementons les operations classiques (intersection, union, complement).

In [14]:
def symbolic_intersection(aut1: SymbolicAutomaton, aut2: SymbolicAutomaton,
                          name: str = "Intersection") -> SymbolicAutomaton:
    """
    Intersection de deux automates symboliques.
    Le predicat resultant est la conjonction des predicats.
    """
    if len(aut1.transitions) != 1 or len(aut2.transitions) != 1:
        raise ValueError("Implemente pour automates a une transition")
    
    result = SymbolicAutomaton(name)
    result.add_state('q0', is_initial=True)
    result.add_state('q1', is_final=True)
    
    # Conjonction des predicats
    _, _, pred1 = aut1.transitions[0]
    _, _, pred2 = aut2.transitions[0]
    combined_pred = And(pred1, pred2)
    
    result.add_transition('q0', 'q1', combined_pred)
    return result

def symbolic_union(aut1: SymbolicAutomaton, aut2: SymbolicAutomaton,
                   name: str = "Union") -> SymbolicAutomaton:
    """
    Union de deux automates symboliques.
    Le predicat resultant est la disjonction des predicats.
    """
    if len(aut1.transitions) != 1 or len(aut2.transitions) != 1:
        raise ValueError("Implemente pour automates a une transition")
    
    result = SymbolicAutomaton(name)
    result.add_state('q0', is_initial=True)
    result.add_state('q1', is_final=True)
    
    # Disjonction des predicats
    _, _, pred1 = aut1.transitions[0]
    _, _, pred2 = aut2.transitions[0]
    combined_pred = Or(pred1, pred2)
    
    result.add_transition('q0', 'q1', combined_pred)
    return result

def symbolic_complement(aut: SymbolicAutomaton,
                        name: str = "Complement") -> SymbolicAutomaton:
    """
    Complement d'un automate symbolique.
    Le predicat resultant est la negation du predicat.
    """
    if len(aut.transitions) != 1:
        raise ValueError("Implemente pour automates a une transition")
    
    result = SymbolicAutomaton(name)
    result.add_state('q0', is_initial=True)
    result.add_state('q1', is_final=True)
    
    # Negation du predicat
    _, _, pred = aut.transitions[0]
    negated_pred = Not(pred)
    
    result.add_transition('q0', 'q1', negated_pred)
    return result

print("Operations sur automates symboliques definies :")
print("  - Intersection (conjonction de predicats)")
print("  - Union (disjonction de predicats)")
print("  - Complement (negation de predicat)")

Operations sur automates symboliques definies :
  - Intersection (conjonction de predicats)
  - Union (disjonction de predicats)
  - Complement (negation de predicat)


### 4.8 Exemple d'Operations

Appliquons les operations sur nos automates.

In [15]:
# Creer deux automates de base
# A1 : Nombres dans [0, 50]
aut1 = SymbolicAutomaton("Range0_50")
aut1.add_state('q0', is_initial=True)
aut1.add_state('q1', is_final=True)
x = Int('x')
aut1.add_transition('q0', 'q1', And(x >= 0, x <= 50))

# A2 : Nombres pairs
aut2 = SymbolicAutomaton("Even")
aut2.add_state('q0', is_initial=True)
aut2.add_state('q1', is_final=True)
aut2.add_transition('q0', 'q1', x % 2 == 0)

# Operations
inter = symbolic_intersection(aut1, aut2, "EvenIn0_50")
union = symbolic_union(aut1, aut2, "InRangeOrEven")
comp = symbolic_complement(aut1, "NotIn0_50")

print("Operations sur automates symboliques\n")
print("A1 : Nombres dans [0, 50]")
print("A2 : Nombres pairs")
print()

# Tests
test_values = [-10, -1, 0, 1, 10, 25, 50, 51, 100]

print("1. INTERSECTION (A1 ∩ A2) : Nombres pairs dans [0, 50]")
print("-" * 60)
for val in test_values:
    if inter.accepts(val):
        print(f"  {val:3d} : ✓ Accepte")

print("\n2. UNION (A1 ∪ A2) : Dans [0, 50] OU pair")
print("-" * 60)
for val in test_values:
    if union.accepts(val):
        print(f"  {val:3d} : ✓ Accepte")

print("\n3. COMPLEMENT (A1^c) : PAS dans [0, 50]")
print("-" * 60)
for val in test_values:
    if comp.accepts(val):
        print(f"  {val:3d} : ✓ Accepte")

Operations sur automates symboliques

A1 : Nombres dans [0, 50]
A2 : Nombres pairs

1. INTERSECTION (A1 ∩ A2) : Nombres pairs dans [0, 50]
------------------------------------------------------------
    0 : ✓ Accepte
   10 : ✓ Accepte
   50 : ✓ Accepte

2. UNION (A1 ∪ A2) : Dans [0, 50] OU pair
------------------------------------------------------------
  -10 : ✓ Accepte
    0 : ✓ Accepte
    1 : ✓ Accepte
   10 : ✓ Accepte
   25 : ✓ Accepte
   50 : ✓ Accepte
  100 : ✓ Accepte

3. COMPLEMENT (A1^c) : PAS dans [0, 50]
------------------------------------------------------------
  -10 : ✓ Accepte
   -1 : ✓ Accepte
   51 : ✓ Accepte
  100 : ✓ Accepte


### Interpretation : Operations sur Automates Symboliques

**Sortie obtenue** : Les operations logiques sur les predicats se traduisent directement en operations sur les langages reconnus.

**Intersection** (A1 ∩ A2) :
- Predicat : $0 \leq x \leq 50 \land x \mod 2 = 0$
- Valeurs acceptees : 0, 2, 4, ..., 48, 50
- Signification : Nombres pairs dans l'intervalle [0, 50]

**Union** (A1 ∪ A2) :
- Predicat : $(0 \leq x \leq 50) \lor (x \mod 2 = 0)$
- Valeurs acceptees : Tous les pairs, plus les impairs dans [0, 50]
- Signification : Soit dans l'intervalle, soit pair (ou les deux)

**Complement** (A1^c) :
- Predicat : $\neg(0 \leq x \leq 50)$
- Valeurs acceptees : $x < 0$ ou $x > 50$
- Signification : Nombres en dehors de l'intervalle [0, 50]

**Point cle** : Les operations sur les automates symboliques correspondent aux operations ensemblistes classiques sur les langages.

## 5. Application - Verification de Proprietes

### 5.1 Probleme de verification

Les automates symboliques sont largement utilises en **verification de model** (model checking) pour prouver des proprietes sur des systemes.

**Exemple** : Systeme de porte avec code
- La porte s'ouvre si le bon code est entre
- Apres 3 essais faux, le systeme se bloque
- On veut verifier que "la porte ne s'ouvre jamais avec un mauvais code"

### 5.2 Modelisation du systeme de porte

Modelisons ce systeme comme un automate symbolique.

In [16]:
# Systeme de securite simplifie
# On verifie qu'un code est dans une plage valide

class SecuritySystem:
    """
    Systeme de verification de code simplifie.
    
    Le code valide est dans une plage secrete [MIN_CODE, MAX_CODE].
    L'automate verifie si un code entre est valide.
    """
    
    def __init__(self, min_code: int, max_code: int):
        self.min_code = min_code
        self.max_code = max_code
        
        # Creer l'automate de verification
        self.automaton = SymbolicAutomaton("SecurityAutomaton")
        self.automaton.add_state('locked', is_initial=True)
        self.automaton.add_state('unlocked', is_final=True)
        
        # Predicat : code doit etre dans la plage valide
        x = Int('code')
        predicate = And(x >= min_code, x <= max_code)
        self.automaton.add_transition('locked', 'unlocked', predicate)
    
    def verify_code(self, code: int) -> bool:
        """Verifie si un code est valide."""
        return self.automaton.accepts(code, variable_name='code')
    
    def is_safe(self, code: int) -> bool:
        """
        Verifie la propriete de securite :
        "Un code hors de la plage valide n'ouvre jamais la porte"
        """
        # Si le code est hors de la plage, il ne doit PAS etre accepte
        outside_range = (code < self.min_code) or (code > self.max_code)
        
        if outside_range:
            accepted = self.verify_code(code)
            return not accepted  # Safe = non accepte
        return True  # Dans la plage, pas de probleme de securite

# Creer un systeme avec code valide dans [1000, 9999]
security = SecuritySystem(1000, 9999)

print("Systeme de securite - Porte a code\n")
print(f"Plage de codes valides : [{security.min_code}, {security.max_code}]")
print()

# Tests de verification
test_codes = [
    (0, False, "Code nul"),
    (999, False, "Juste avant la plage"),
    (1000, True, "Borne inferieure"),
    (5000, True, "Code moyen"),
    (9999, True, "Borne superieure"),
    (10000, False, "Juste apres la plage"),
    (99999, False, "Code trop grand"),
]

print("Tests de verification :")
print("-" * 60)
for code, should_be_valid, desc in test_codes:
    is_valid = security.verify_code(code)
    is_safe = security.is_safe(code)
    
    status = "✓ Valide" if is_valid else "✗ Invalide"
    safety = "✓ Secure" if is_safe else "✗ UNSECURE"
    
    print(f"  Code {code:6d} ({desc:20s}): {status:12s} | {safety}")

Systeme de securite - Porte a code

Plage de codes valides : [1000, 9999]

Tests de verification :
------------------------------------------------------------
  Code      0 (Code nul            ): ✗ Invalide   | ✓ Secure
  Code    999 (Juste avant la plage): ✗ Invalide   | ✓ Secure
  Code   1000 (Borne inferieure    ): ✓ Valide     | ✓ Secure
  Code   5000 (Code moyen          ): ✓ Valide     | ✓ Secure
  Code   9999 (Borne superieure    ): ✓ Valide     | ✓ Secure
  Code  10000 (Juste apres la plage): ✗ Invalide   | ✓ Secure
  Code  99999 (Code trop grand     ): ✗ Invalide   | ✓ Secure


### Interpretation : Verification de Proprietes

**Sortie obtenue** : Le systeme verifie correctement les codes et maintient la propriete de securite.

**Propriete de securite** : "Un code hors de la plage valide n'ouvre jamais la porte"

| Code | Dans la plage | Valide | Secure |
|------|---------------|--------|--------|
| 0 | Non | Non | Oui (rejete) |
| 999 | Non | Non | Oui (rejete) |
| 1000 | Oui | Oui | Oui (accepte) |
| 5000 | Oui | Oui | Oui (accepte) |
| 9999 | Oui | Oui | Oui (accepte) |
| 10000 | Non | Non | Oui (rejete) |

**Points cles** :
1. L'automate symbolique modelise le comportement du systeme
2. La propriete de securite est verifiee par test sur les predicats
3. Tous les codes hors plage sont rejetes (systeme securise)

> **En pratique** : Model checking utilise des techniques plus avancees (BDDs, SAT/SMT solvers) pour verifier des systemes avec millions d'etats.

### 5.3 Invariants d'etat

Un **invariant** est une propriete qui doit toujours etre vraie dans tous les etats accessibles du systeme.

In [17]:
# Exemple d'invariant : compteur borne

class BoundedCounter:
    """
    Compteur avec invariant : 0 <= value <= MAX
    """
    
    def __init__(self, max_value: int):
        self.max_value = max_value
        self.value = 0
    
    def increment(self) -> bool:
        """
        Incremente le compteur si possible.
        Retourne True si l'invariant est maintenu.
        """
        old_value = self.value
        new_value = old_value + 1
        
        # Verifier l'invariant sur la nouvelle valeur
        x = Int('x')
        invariant = And(x >= 0, x <= self.max_value)
        
        # Creer un solver pour tester
        s = Solver()
        s.add(invariant)
        s.add(x == new_value)
        
        if s.check() == sat:
            self.value = new_value
            return True
        else:
            # L'invariant serait viole
            return False
    
    def decrement(self) -> bool:
        """
        Decremente le compteur si possible.
        Retourne True si l'invariant est maintenu.
        """
        old_value = self.value
        new_value = old_value - 1
        
        # Verifier l'invariant
        x = Int('x')
        invariant = And(x >= 0, x <= self.max_value)
        
        s = Solver()
        s.add(invariant)
        s.add(x == new_value)
        
        if s.check() == sat:
            self.value = new_value
            return True
        else:
            return False

# Test du compteur borne
counter = BoundedCounter(max_value=5)

print("Compteur borne avec invariant : 0 <= value <= 5\n")
print("Operations :")
print("-" * 50)

# Incrementer jusqu'a la limite
for i in range(7):
    success = counter.increment()
    status = "✓" if success else "✗ Echec (invariant viole)"
    print(f"  Increment {i+1}: value={counter.value} {status}")

print()

# Decrementer jusqu'a la limite
for i in range(7):
    success = counter.decrement()
    status = "✓" if success else "✗ Echec (invariant viole)"
    print(f"  Decrement {i+1}: value={counter.value} {status}")

Compteur borne avec invariant : 0 <= value <= 5

Operations :
--------------------------------------------------
  Increment 1: value=1 ✓
  Increment 2: value=2 ✓
  Increment 3: value=3 ✓
  Increment 4: value=4 ✓
  Increment 5: value=5 ✓
  Increment 6: value=5 ✗ Echec (invariant viole)
  Increment 7: value=5 ✗ Echec (invariant viole)

  Decrement 1: value=4 ✓
  Decrement 2: value=3 ✓
  Decrement 3: value=2 ✓
  Decrement 4: value=1 ✓
  Decrement 5: value=0 ✓
  Decrement 6: value=0 ✗ Echec (invariant viole)
  Decrement 7: value=0 ✗ Echec (invariant viole)


### Interpretation : Invariants d'etat

**Sortie obtenue** : Le compteur maintient l'invariant $0 \leq value \leq 5$.

**Invariant** : Une propriete qui doit etre vraie dans tous les etats accessibles.

**Comportement observe** :
- Les 5 premiers incrementations reussissent (value: 0 → 1 → 2 → 3 → 4 → 5)
- La 6e incrementation echoue (value resterait a 6, > 5)
- Les 6 premieres decrementation reussissent (value: 5 → 4 → 3 → 2 → 1 → 0)
- La 7e decrementation echoue (value serait -1, < 0)

**Points cles** :
1. L'invariant est exprime comme un predicat Z3
2. Chaque operation verifie que l'invariant est maintenu
3. Le solver Z3 determine si la nouvelle valeur satisfait l'invariant

> **Application** : Cette technique est utilisee dans les outils de model checking (SPIN, NuSMV) pour verifier des systemes concurrents et distribues.

## 6. Lien avec Sudoku

### 6.1 Sudoku comme probleme d'automates

Le Sudoku peut etre modelise comme un automate symbolique :

- **Etats** : Configurations partielles ou completes de la grille
- **Transitions** : Placement d'un chiffre dans une case vide
- **Predicats** : Contraintes Sudoku (ligne, colonne, bloc)
- **Etats finaux** : Grilles completes et valides

**Contraintes comme predicats Z3** :
- TousDistinct(ligne[i])
- TousDistinct(colonne[j])
- TousDistinct(bloc[k])
- Chaque case dans $[1, 9]$

### 6.2 Exemple simplifie - Mini-Sudoku 2x2

Illustrons avec un Sudoku 2x2 (4 cases, chiffres 1-2).

In [18]:
# Mini-Sudoku 2x2 comme automate symbolique

class MiniSudokuAutomaton:
    """
    Automate symbolique pour Mini-Sudoku 2x2.
    
    Grille 2x2 avec chiffres 1-2.
    Contraintes : lignes et colonnes doivent avoir des chiffres distincts.
    """
    
    def __init__(self):
        self.grid = [[0, 0], [0, 0]]  # 0 = vide
    
    def is_valid_placement(self, row: int, col: int, value: int) -> bool:
        """
        Verifie si le placement est valide (contraintes Sudoku).
        Utilise Z3 pour exprimer les contraintes.
        """
        # Verifier que la case est vide
        if self.grid[row][col] != 0:
            return False
        
        # Verifier la plage de valeur
        if value not in [1, 2]:
            return False
        
        # Simuler le placement
        self.grid[row][col] = value
        
        # Verifier les contraintes avec Z3
        s = Solver()
        
        # Variables pour les cases
        cells = [Int(f'c_{i}_{j}') for i in range(2) for j in range(2)]
        
        # Contrainte : chaque case doit etre 1 ou 2 (ou 0 si vide)
        for i in range(2):
            for j in range(2):
                if self.grid[i][j] != 0:
                    s.add(cells[i*2 + j] == self.grid[i][j])
                else:
                    s.add(Or(cells[i*2 + j] == 1, cells[i*2 + j] == 2))
        
        # Contrainte : lignes distinctes
        s.add(Distinct([cells[0], cells[1]]))  # Ligne 0
        s.add(Distinct([cells[2], cells[3]]))  # Ligne 1
        
        # Contrainte : colonnes distinctes
        s.add(Distinct([cells[0], cells[2]]))  # Colonne 0
        s.add(Distinct([cells[1], cells[3]]))  # Colonne 1
        
        # Verifier la satisfiabilite
        valid = s.check() == sat
        
        # Revertir le placement
        self.grid[row][col] = 0
        
        return valid
    
    def place(self, row: int, col: int, value: int) -> bool:
        """Place une valeur si valide."""
        if self.is_valid_placement(row, col, value):
            self.grid[row][col] = value
            return True
        return False
    
    def display(self):
        """Affiche la grille."""
        print("Grille Mini-Sudoku 2x2 :")
        print(f"  {self.grid[0][0]} {self.grid[0][1]}")
        print(f"  {self.grid[1][0]} {self.grid[1][1]}")

# Test
mini_sudoku = MiniSudokuAutomaton()

print("Mini-Sudoku 2x2 avec Z3\n")
mini_sudoku.display()
print()

# Essayer de placer des valeurs
placements = [
    (0, 0, 1, "Premier placement"),
    (0, 1, 1, "Essayer de dupliquer dans la ligne"),
    (0, 1, 2, "Valeur correcte"),
    (1, 0, 2, "Essayer de dupliquer dans la colonne"),
    (1, 0, 1, "Valeur correcte"),
    (1, 1, 1, "Seule valeur possible"),
]

for row, col, val, desc in placements:
    success = mini_sudoku.place(row, col, val)
    status = "✓" if success else "✗"
    print(f"{status} Place ({row},{col})={val} : {desc}")
    mini_sudoku.display()
    print()

Mini-Sudoku 2x2 avec Z3

Grille Mini-Sudoku 2x2 :
  0 0
  0 0

✓ Place (0,0)=1 : Premier placement
Grille Mini-Sudoku 2x2 :
  1 0
  0 0

✗ Place (0,1)=1 : Essayer de dupliquer dans la ligne
Grille Mini-Sudoku 2x2 :
  1 0
  0 0

✓ Place (0,1)=2 : Valeur correcte
Grille Mini-Sudoku 2x2 :
  1 2
  0 0

✓ Place (1,0)=2 : Essayer de dupliquer dans la colonne
Grille Mini-Sudoku 2x2 :
  1 2
  2 0

✗ Place (1,0)=1 : Valeur correcte
Grille Mini-Sudoku 2x2 :
  1 2
  2 0

✓ Place (1,1)=1 : Seule valeur possible
Grille Mini-Sudoku 2x2 :
  1 2
  2 1



### Interpretation : Mini-Sudoku comme Automate

**Sortie obtenue** : Le Mini-Sudoku utilise des predicats Z3 pour valider les placements.

**Evolution de la grille** :

| Etape | Placement | Valide | Raison |
|-------|-----------|--------|--------|
| 1 | (0,0)=1 | Oui | Case vide, pas de conflit |
| 2 | (0,1)=1 | Non | Conflit ligne (deux 1) |
| 3 | (0,1)=2 | Oui | Case vide, pas de conflit |
| 4 | (1,0)=2 | Non | Conflit colonne (deux 2) |
| 5 | (1,0)=1 | Oui | Case vide, pas de conflit |
| 6 | (1,1)=1 | Non | Conflit ligne ET colonne |

**Point cle** : Le predicat `Distinct` de Z3 verifie que toutes les variables d'une liste ont des valeurs differentes, ce qui modelise directement les contraintes Sudoku.

> **Pour aller plus loin** : Voir [Sudoku-12-SymbolicAutomata](../../Sudoku/Sudoku-12-SymbolicAutomata.ipynb) pour une application complete.

## 7. Automata.Net - Pourquoi Pas ?

### 7.1 La librairie Automata.Net

**Automata.Net** est une librairie C# pour les automates finis, developpee vers 2017-2018.

**Pourquoi nous ne l'utilisons pas** :

| Raison | Detail |
|--------|--------|
| **Obsolete** | Plus de mises a jour depuis 2017-2018 |
| **Bug non resolu** | Issue #6 ouverte depuis des annees sans correction |
| **Alphabet fini uniquement** | Pas de support pour predicats symboliques |
| **Limitation C#** | Integration complexe avec Jupyter .NET Interactive |

### 7.2 Notre approche alternative

Au lieu d'Automata.Net, nous utilisons :

1. **automata-lib** (Python)
   - Pour les automates finis classiques
   - API simple, bien maintenue
   - Operations : union, intersection, complement

2. **Z3** (Python et C#)
   - Pour les automates symboliques
   - Predicats logiques puissants
   - SMT solver etendu

3. **Implementation personnalisee**
   - Classe `SymbolicAutomaton` ce notebook
   - Adaptation aux besoins specifiques
   - Flexibilite pour l'extension

## 8. Resume

### Concepts cles

| Concept | Definition |
|---------|------------|
| **Automate fini** | $Q, \Sigma, \delta, q_0, F$ avec alphabet fini |
| **DFA** | Automate deterministe (une transition par etat/symbole) |
| **NFA** | Automate non-deterministe (0, 1 ou plusieurs transitions) |
| **Automate symbolique** | Transitions avec predicats logiques |
| **Predicat** | Formule logique sur l'alphabet (ex: $x > 0$) |

### Operations

| Operation | Automate fini | Automate symbolique |
|-----------|---------------|---------------------|
| Union | $L_1 \cup L_2$ | $\phi_1 \lor \phi_2$ |
| Intersection | $L_1 \cap L_2$ | $\phi_1 \land \phi_2$ |
| Complement | $\Sigma^* \setminus L$ | $\neg \phi$ |

### Outils

| Outil | Usage | Avantages |
|--------|-------|-----------|
| automata-lib | Automates finis classiques | Python, API simple |
| Z3 | Automates symboliques | Predicats, SMT solver |

### Applications
- Verification de model (model checking)
- Analyse de programmes (symbolic execution)
- Resolution de contraintes (CSP, Sudoku)
- Verification de protocoles

### Pour aller plus loin

- **Sudoku** : [Sudoku-12-SymbolicAutomata](../../Sudoku/Sudoku-12-SymbolicAutomata.ipynb)
- **SymbolicAI** : [SymbolicAI/SymbolicAI](../../SymbolicAI/SymbolicAI/README.md)
- **Applications** : [App-1-NQueens](../Applications/App-1-NQueens.ipynb)

## 9. Exercices

### Exercice 1 : Automate pour multiples de 3

**Tache** : Implementez un automate symbolique qui reconnait les nombres divisibles par 3.

**Indice** : Utilisez le predicat `x % 3 == 0`.

In [19]:
# Exercice 1 : Automate pour multiples de 3

# Implementer l'automate
automaton_mult3 = SymbolicAutomaton("Mult3Automaton")

# Ajouter les etats
automaton_mult3.add_state('q0', is_initial=True)
automaton_mult3.add_state('q1', is_final=True)

# Definir le predicat pour multiples de 3
x = Int('x')
predicate = x % 3 == 0

# Ajouter la transition
automaton_mult3.add_transition('q0', 'q1', predicate)

# Tests
test_values = list(range(-10, 21))
print("Exercice 1 : Multiples de 3\n")
print(f"Predicat : {predicate}")
print("\nValeurs acceptees :")
for val in test_values:
    if automaton_mult3.accepts(val):
        print(f"  {val}")

# Resume
print(f"\nTotal valeurs acceptees dans [-10, 20] : {sum(1 for v in test_values if automaton_mult3.accepts(v))}")
print("Attendu : 11 valeurs (-9, -6, -3, 0, 3, 6, 9, 12, 15, 18)")

Exercice 1 : Multiples de 3

Predicat : x%3 == 0

Valeurs acceptees :
  -9
  -6
  -3
  0
  3
  6
  9
  12
  15
  18

Total valeurs acceptees dans [-10, 20] : 10
Attendu : 11 valeurs (-9, -6, -3, 0, 3, 6, 9, 12, 15, 18)


### Exercice 2 : Automate pour nombres impairs positifs

**Tache** : Implementez un automate symbolique qui reconnait les nombres impairs strictement positifs.

**Predicat** : $x > 0 \land x \mod 2 \neq 0$

In [20]:
# Exercice 2 : Nombres impairs positifs

# Implementer l'automate
automaton_odd_pos = SymbolicAutomaton("OddPositiveAutomaton")

# Ajouter les etats
automaton_odd_pos.add_state('q0', is_initial=True)
automaton_odd_pos.add_state('q1', is_final=True)

# Definir le predicat : x > 0 ET x est impair
x = Int('x')
# x % 2 == 1 signifie "x est impair" (reste de la division par 2 est 1)
predicate = And(x > 0, x % 2 == 1)

# Ajouter la transition
automaton_odd_pos.add_transition('q0', 'q1', predicate)

# Tests
test_values = list(range(-10, 21))
print("Exercice 2 : Nombres impairs positifs\n")
print(f"Predicat : {predicate}")
print("\nValeurs acceptees :")
for val in test_values:
    if automaton_odd_pos.accepts(val):
        print(f"  {val}")

# Resume
print(f"\nTotal valeurs acceptees dans [-10, 20] : {sum(1 for v in test_values if automaton_odd_pos.accepts(v))}")
print("Attendu : 10 valeurs (1, 3, 5, 7, 9, 11, 13, 15, 17, 19)")

Exercice 2 : Nombres impairs positifs

Predicat : And(x > 0, x%2 == 1)

Valeurs acceptees :
  1
  3
  5
  7
  9
  11
  13
  15
  17
  19

Total valeurs acceptees dans [-10, 20] : 10
Attendu : 10 valeurs (1, 3, 5, 7, 9, 11, 13, 15, 17, 19)


### Exercice 3 : Comparaison Fini vs Symbolique

**Tache** : Comparez la taille d'un automate fini et d'un automate symbolique pour reconnaitre les entiers pairs entre 100 et 200.

**Questions** :
1. Combien d'etats necessite l'automate fini ?
2. Combien d'etats necessite l'automate symbolique ?
3. Quel est le predicat de l'automate symbolique ?

In [21]:
# Exercice 3 : Comparaison

print("Exercice 3 : Comparaison Fini vs Symbolique\n")
print("Tache : Reconnaître les entiers pairs entre 100 et 200")
print()

# 1. Automate fini classique
print("1. Automate fini classique :")
print("   Analyse detaillee :")
print("   - Plage : [100, 200]")
print("   - Nombres pairs : 100, 102, 104, ..., 198, 200")
print("   - Nombre de valeurs acceptees = ((200 - 100) / 2) + 1 = 51")
print()
print("   Pour un automate fini classique :")
print("   - Option A (explicite) : 51 etats (un par valeur paire acceptee)")
print("   - Option B (minimal) : 3 etats (etat initial, plage etrangere, plage interieure)")
print("   - Avec un alphabet de 101 symboles {100, 101, ..., 200}")
print()

# 2. Automate symbolique
print("2. Automate symbolique :")
automaton_comp = SymbolicAutomaton("ComparisonAutomaton")
automaton_comp.add_state('q0', is_initial=True)
automaton_comp.add_state('q1', is_final=True)
x = Int('x')
predicate = And(x >= 100, x <= 200, x % 2 == 0)
automaton_comp.add_transition('q0', 'q1', predicate)

print(f"   Nombre d'etats : {len(automaton_comp.states)}")
print(f"   Predicat : {predicate}")
print()

# 3. Comparaison
print("3. Comparaison :")
print("   Ratio d'etats (Fini explicite / Symbolique) : 51 / 2 = 25.5x")
print()
print("   Avantages de l'automate symbolique :")
print("   - Taille constante (2 etats) independante de la plage")
print("   - Fonctionne pour n'importe quel entier (alphabet infini)")
print("   - Predicat lisible et maintenable")
print()
print("   Inconvenients :")
print("   - Necessite un SMT solver (Z3)")
print("   - Verification plus lente que test direct")

# Verification
print("\n   Tests de verification :")
test_vals = [99, 100, 101, 150, 199, 200, 201]
for val in test_vals:
    accepted = automaton_comp.accepts(val)
    status = "[OK]" if accepted else "[X]"
    print(f"     {status} {val:3d} : {'Accepte' if accepted else 'Rejete'}")

Exercice 3 : Comparaison Fini vs Symbolique

Tache : Reconnaître les entiers pairs entre 100 et 200

1. Automate fini classique :
   Analyse detaillee :
   - Plage : [100, 200]
   - Nombres pairs : 100, 102, 104, ..., 198, 200
   - Nombre de valeurs acceptees = ((200 - 100) / 2) + 1 = 51

   Pour un automate fini classique :
   - Option A (explicite) : 51 etats (un par valeur paire acceptee)
   - Option B (minimal) : 3 etats (etat initial, plage etrangere, plage interieure)
   - Avec un alphabet de 101 symboles {100, 101, ..., 200}

2. Automate symbolique :
   Nombre d'etats : 2
   Predicat : And(x >= 100, x <= 200, x%2 == 0)

3. Comparaison :
   Ratio d'etats (Fini explicite / Symbolique) : 51 / 2 = 25.5x

   Avantages de l'automate symbolique :
   - Taille constante (2 etats) independante de la plage
   - Fonctionne pour n'importe quel entier (alphabet infini)
   - Predicat lisible et maintenable

   Inconvenients :
   - Necessite un SMT solver (Z3)
   - Verification plus lente que t

---

**Navigation** : [Index](../README.md) | [Suivant >>](../Applications/App-1-NQueens.ipynb)

**Series connexes** : 
- [SymbolicAI/SymbolicAI/Sudoku-4-Z3](../../SymbolicAI/SymbolicAI/Sudoku-4-Z3.ipynb) - Bases de Z3
- [Sudoku-12-SymbolicAutomata](../../Sudoku/Sudoku-12-SymbolicAutomata.ipynb) - Sudoku solver par automates symboliques